LIBRERIAS

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import time
from datetime import datetime, timedelta
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options

Funciones

In [3]:
def es_entero(s):
    try:
        int(s)
        return True
    except ValueError:
        return False
    
def es_partido_dobles(linea):
    posibles_nombres = linea[2:7]

    nombres_detectados = [
        n for n in posibles_nombres
        if '.' in n and len(n.split()) <= 2 and 'atp' not in n.lower() and 'wta' not in n.lower()
    ]

    return len(nombres_detectados) >= 4

ÚLTIMA VERSIÓN DEL CÓDIGO

In [21]:
try:
    driver.quit()
except:
    pass  # Por si driver aún no estaba creado

options = Options()
#options.add_argument("--headless")

driver = webdriver.Chrome(options=options)
driver.get("https://www.sofascore.com/tennis")

try:
    wait = WebDriverWait(driver, 10)

    # Eliminar overlays
    def wait_overlay_to_disappear(driver, wait, class_name="fc-dialog-overlay", timeout=10):
        try:
            wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, class_name)))
        except TimeoutException:
            driver.execute_script("""
                let overlays = document.getElementsByClassName(arguments[0]);
                while(overlays.length > 0) {
                    overlays[0].parentNode.removeChild(overlays[0]);
                }
            """, class_name)
            time.sleep(1)

    #Esto que está comentado es para eliminar pestañas emergentes de manera automática (no sirve para captcha)
    """
    # Aceptar cookies u otros popups
    buttons = driver.find_elements(By.TAG_NAME, "button")
    print(len(buttons), buttons)
    for i in range(len(buttons)):
        try:
            buttons = driver.find_elements(By.TAG_NAME, "button")
            btn = buttons[i]
            text = btn.text.lower()
            if any(word in text for word in ["aceptar", "accept", "consent", "confirm", "consentir", "si"]):
                wait_overlay_to_disappear(driver, wait)
                wait.until(EC.element_to_be_clickable(btn))
                driver.execute_script("arguments[0].scrollIntoView();", btn)
                time.sleep(10)
                btn.click()
                break
        except StaleElementReferenceException:
            continue
        except Exception:
            continue"""

    # Esperar manualmente a que se resuelva el CAPTCHA
    input("Resuelve el CAPTCHA (y otras cosas) manualmente en el navegador y presiona Enter aquí para continuar...")

    time.sleep(2)

    # Variables
    base_url = "https://www.sofascore.com/es/tenis"
    fecha_actual = datetime(2025, 4, 24)
    datos = []

    for i in range(3):
        fecha = fecha_actual - timedelta(days=i)
        fecha_str = fecha.strftime('%Y-%m-%d')
        url = f"{base_url}/{fecha_str}"
        print("Navegando a URL:", url)
        driver.get(url)
        print("dentro")

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.Box.klGMtt")))
        pista = driver.find_elements(By.CSS_SELECTOR, "div.Box.klGMtt")

        enlaces_partidos = []
        torneo_actual = {}

        #print("Número de pistas", len(pista))
        for p in pista:
            texto = p.text.lower()
            lineas = p.text.split("\n")

            # Detectar y almacenar torneo actual
            if not any(x in texto for x in ["matches"]) and any(x in texto for x in ["grand slam", "atp", "wta", "open", 'challenger', 'itf', 'utr']):
                if len(lineas) == 3:
                    tipo = lineas[0]
                    if tipo == 'Grand Slam':
                        lugar_partes = [s.strip() for s in lineas[1].split(",")]
                        nombre = lugar_partes[0]
                        ciudad = lugar_partes[1] if len(lugar_partes) > 1 else ""
                        pais = lugar_partes[2] if len(lugar_partes) > 2 else ""
                        especificacion = lugar_partes[3] if len(lugar_partes) > 3 else "Singles"
                        superficie = lineas[2]
                        if ciudad in ['Boys', 'Girls'] or pais == 'Doubles':
                            ciudad = 'Paris'
                            pais = 'France'
                            especificacion = 'Doubles'
                    else:
                        tipo = lineas[0]
                        lugar_partes = [s.strip() for s in lineas[1].split(",")]
                        ciudad = lugar_partes[0]
                        pais = lugar_partes[1] if len(lugar_partes) > 1 else ""
                        especificacion = lugar_partes[2] if len(lugar_partes) > 2 else "Singles"
                        superficie = lineas[2]
                        nombre = ciudad

                    torneo_actual = {
                        "tipo": tipo,
                        "nombre": nombre,
                        "ciudad": ciudad,
                        "pais": pais,
                        "especificacion": especificacion,
                        "superficie": superficie
                    }

                    #print("Torneo detectado:", torneo_actual)

            # Extraer enlace si es un partido
            if not any(x in texto for x in ["grand slam", "atp", "wta", "open", 'challenger', 'itf', 'utr', 'matches']) and texto.strip() != "":
                links = p.find_elements(By.TAG_NAME, "a")
                if links:
                    href = links[0].get_attribute("href")
                    enlaces_partidos.append((href, torneo_actual.copy()))  # Guardamos una copia del torneo actual

        print("Número de enlaces", len(enlaces_partidos))
        
        for href, torneo_info in enlaces_partidos:
            try:
                driver.get(href)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.d_flex.flex-d_column.gap_sm.p_sm")))
                est = driver.find_elements(By.CSS_SELECTOR, "div.d_flex.flex-d_column.gap_sm.p_sm")

                cuotaA = cuotaB = fecha_texto = dia = hora = dur = genero = antes = regla = setA = setB = nSets = "-"
                cancelado = "no"
                h2h1 = h2h2 = 0
                duraciones = []
                jugadores = []
                ranking = []
                sets = []

                for e in est:
                    lineas = e.text.split("\n")
                    #print("Número de lineas", len(lineas))
                    #print(lineas)  # lineas equivale a un partido

                    for i, line in enumerate(lineas):
                        #print(i, line)

                        if 'odds' in line.lower() or 'cuotas' in line.lower():
                            if i >= 3:
                                cuotaA = lineas[i - 3]
                                cuotaB = lineas[i - 1]

                        if "ft" in line.lower() or "retired" in line.lower() or 'final' in line.lower():
                            duraciones.append(lineas[i].strip())

                        if es_partido_dobles(lineas):
                            if "." in line and i >= 2 and i <= 7 and not ("atp" in line.lower() or "wta" in line.lower()): 
                                jugadores.append(line)
                        else:
                            if 'wta' in line.lower():
                                if i >= 1:
                                    jugadores.append(lineas[i - 1])
                                ranking.append(lineas[i])
                                antes = i
                                if genero == '-':
                                    genero = 'femenino'

                            if 'atp' in line.lower():
                                if i >= 1:
                                    jugadores.append(lineas[i - 1])
                                ranking.append(lineas[i])
                                antes = int(i)
                                if genero == '-':
                                    genero = 'masculino'

                        if "ft" in line.lower() or "retired" in line.lower() or 'final' in line.lower():
                            es, dur = lineas[i].split(" - ")

                        if "cancelado" in line.lower() or "cancelled" in line.lower() or "canceled" in line.lower():
                            cancelado = "si"

                        setA = lineas[len(lineas) - 2]
                        setB = lineas[len(lineas) - 1]

                        if len(lineas) > 0 and "/" in lineas[0]:
                            dia = lineas[0]
                        if len(lineas) > 1 and ":" in lineas[1]:
                            hora = lineas[1]

                # Determinar ganador
                ganador = -1
                try:
                    if setA.isdigit() and setB.isdigit():
                        if int(setA) > int(setB):
                            ganador = 0
                        elif int(setB) > int(setA):
                            ganador = 1
                except:
                    pass


                # Guardar en lista de diccionarios
                datos.append({
                    "tipo_torneo": torneo_info.get("tipo", ""),
                    "nombre_torneo": torneo_info.get("nombre", ""),
                    "ciudad_torneo": torneo_info.get("ciudad", ""),
                    "pais_torneo": torneo_info.get("pais", ""),
                    "especificacion_torneo": torneo_info.get("especificacion", ""),
                    "superficie_torneo": torneo_info.get("superficie", ""),
                    "estado_partido": es if 'es' in locals() else "-",
                    "jugadorA": jugadores[0] if len(jugadores) > 0 else "-",
                    "jugadorB": jugadores[1] if len(jugadores) > 1 else "-",
                    "jugadorA2": jugadores[2] if len(jugadores) > 2 else "-",
                    "jugadorB2": jugadores[3] if len(jugadores) > 3 else "-",
                    "cuotaA": cuotaA,
                    "cuotaB": cuotaB,
                    "duracion": dur,
                    "duracion_sets": " | ".join(duraciones) if duraciones else "-",
                    "setsA": setA,
                    "setsB": setB,
                    "ranking_jugadorA": ranking[0] if len(ranking) > 0 else "-",
                    "ranking_jugadorB": ranking[1] if len(ranking) > 1 else "-",
                    "genero": genero,
                    "dia": dia,
                    "hora": hora,
                    "cancelado": cancelado,
                    "ganador": ganador
                })

            except Exception as e:
                print(f"Error al procesar {href}: {e}")

finally:
    driver.quit()
    # Exportar a CSV
    df = pd.DataFrame(datos)
    display(df)
    #df.to_csv("partidos_tenis2.csv", index=False)
    print("Datos guardados de", fecha_actual)

Navegando a URL: https://www.sofascore.com/es/tenis/2025-04-24
dentro
Número de enlaces 27
Navegando a URL: https://www.sofascore.com/es/tenis/2025-04-23
dentro
Número de enlaces 27
Navegando a URL: https://www.sofascore.com/es/tenis/2025-04-22
dentro
Número de enlaces 25


,tipo_torneo,nombre_torneo,ciudad_torneo,pais_torneo,especificacion_torneo,superficie_torneo,estado_partido,jugadorA,jugadorB,jugadorA2,...,duracion_sets,setsA,setsB,ranking_jugadorA,ranking_jugadorB,genero,dia,hora,cancelado,ganador
0,ATP 1000,Madrid,Madrid,Spain,Singles,Tierra batida,Final,A. Vukic,K. Nishikori,-,...,Final - 1h 59m,1,2,ATP 80.,ATP 58.,masculino,24/4/2025,11:00,no,1
1,ATP 1000,Madrid,Madrid,Spain,Singles,Tierra batida,Final,D. Dzumhur,M. Bellucci,-,...,Final - 2h 34m,2,1,ATP 69.,ATP 68.,masculino,24/4/2025,11:00,no,0
2,ATP 1000,Madrid,Madrid,Spain,Singles,Tierra batida,Final,D. Goffin,A. Muller,-,...,-,1,1,ATP 47.,ATP 39.,masculino,24/4/2025,11:00,no,-1
3,ATP 1000,Madrid,Madrid,Spain,Singles,Tierra batida,Final,T. Etcheverry,H. Medjedovic,-,...,Final - 3h 16m,2,1,ATP 53.,ATP 74.,masculino,24/4/2025,11:00,no,0
4,ATP 1000,Madrid,Madrid,Spain,Singles,Tierra batida,Final,Z. Bergs,Y. Nishioka,-,...,-,5.00,Cuotas adicionales,ATP 50.,ATP 77.,masculino,-,-,si,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,ATP 1000,Madrid,Madrid,Spain,Qualifying,Tierra batida,Final,T. Monteiro,E. Quinn,-,...,Final - 3h 9m,1,2,ATP 131.,ATP 106.,masculino,22/4/2025,16:50,no,1
75,ATP 1000,Madrid,Madrid,Spain,Qualifying,Tierra batida,Final,K. Majchrzak,E. Moller,-,...,Final - 2h 14m,1,2,ATP 96.,ATP 112.,masculino,22/4/2025,17:25,no,1
76,ATP 1000,Madrid,Madrid,Spain,Qualifying,Tierra batida,Final,R. Hijikata,F. Fognini,-,...,Final - 2h 48m,1,2,ATP 78.,ATP 107.,masculino,22/4/2025,18:40,no,1
77,WTA 1000,Madrid,Madrid,Spain,Qualifying,Tierra batida,Final,A. Blinkova,K. Quevedo,-,...,Final - 1h 54m,2,0,WTA 62.,WTA 248.,femenino,22/4/2025,11:00,no,0


Datos guardados de 2025-04-24 00:00:00


Nuevo dataset

In [ ]:
df.to_csv("data/partidos_tenis_04_27-04_25.csv", index=False)
datos = pd.read_csv("data/partidos_tenis_04_27-04_25.csv")

,tipo_torneo,nombre_torneo,ciudad_torneo,pais_torneo,especificacion_torneo,superficie_torneo,estado_partido,jugadorA,jugadorB,jugadorA2,...,duracion_sets,setsA,setsB,ranking_jugadorA,ranking_jugadorB,genero,dia,hora,cancelado,ganador
50,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,A. Rinderknech,C. Ruud,-,...,FT - 1h 13m,0,2,ATP 75.,ATP 8.,masculino,25/04/2025,11:00,no,1
51,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,B. Bonzi,H. Hurkacz,-,...,FT - 1h 26m,2,0,ATP 60.,ATP 28.,masculino,25/04/2025,11:00,no,0
52,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,F. Cerundolo,H. Mayot,-,...,FT - 1h 29m,2,0,ATP 18.,ATP 151.,masculino,25/04/2025,11:00,no,0
53,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,F. Comesaña,A. Fils,-,...,FT - 1h 45m,2,0,ATP 64.,ATP 14.,masculino,25/04/2025,11:00,no,0
55,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,D. Medvedev,L. Djere,-,...,-,"25 Apr 2025, 12:10",Walkover,ATP 11.,ATP 59.,masculino,25/04/2025,12:10,no,-1
56,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,J. Cerundolo,F. Auger-Aliassime,-,...,FT - 1h 53m,2,0,ATP 109.,ATP 27.,masculino,25/04/2025,12:35,no,0
57,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,A. Popyrin,A. Bublik,-,...,FT - 1h 50m,0,2,ATP 25.,ATP 62.,masculino,25/04/2025,12:40,no,1
58,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,S. Korda,F. Cinà,-,...,FT - 1h 41m,2,1,ATP 23.,ATP 294.,masculino,25/04/2025,13:05,no,0
59,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,E. Quinn,J. Mensik,-,...,FT - 1h 20m,0,2,ATP 106.,ATP 19.,masculino,25/04/2025,14:00,no,1
60,ATP 1000,Madrid,Madrid,Spain,Singles,Red clay,FT,A. Zverev,R. Bautista Agut,-,...,FT - 1h 10m,2,0,ATP 3.,ATP 49.,masculino,25/04/2025,14:20,no,0


Pruebas

In [6]:
for i in datos.columns:
    print(datos[i].value_counts())

tipo_torneo
ATP 1000         31
WTA 1000         15
Challenger 75     8
Challenger        8
Name: count, dtype: int64
nombre_torneo
Madrid        46
Guangzhou      8
Mauthausen     5
Ostrava        3
Name: count, dtype: int64
ciudad_torneo
Madrid        46
Guangzhou      8
Mauthausen     5
Ostrava        3
Name: count, dtype: int64
pais_torneo
Spain             46
China              8
Austria            5
Czech Republic     3
Name: count, dtype: int64
especificacion_torneo
Singles    45
Doubles    17
Name: count, dtype: int64
superficie_torneo
Tierra batida    54
Dura outdoor      8
Name: count, dtype: int64
estado_partido
Final    62
Name: count, dtype: int64
jugadorA
-                      3
F. Tiafoe              2
J. Draper              2
G. Dimitrov            2
M. Keys                2
A. de Minaur           2
M. Uchijima            2
A. Sabalenka           2
M. Arevalo-Gonzalez    2
M. Andreeva            2
S. Cirstea             2
A. Erler               1
J. Salisbury          